# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [193]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [194]:
file_path = "../WeatherPy/output_data/cities.csv"

cities_df = pd.read_csv(file_path)

cities_df['Date'] = cities_df['Date'].astype(int)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chuy,-33.6971,-53.4616,77,68,2,7.61,UY,1611953189
1,mataura,-46.1927,168.8643,48,80,29,8.01,NZ,1611953189
2,lebu,-37.6167,-73.6500,58,94,98,3.62,CL,1611953189
3,tymovskoye,50.8456,142.6550,3,91,100,7.52,RU,1611953190
4,kinshasa,-4.3246,15.3215,82,69,20,4.61,CD,1611953190
...,...,...,...,...,...,...,...,...,...
573,gisborne,-38.6533,178.0042,60,71,90,11.01,NZ,1611953314
574,kazachka,53.3644,38.0694,26,97,100,15.75,RU,1611953314
575,walvis bay,-22.9575,14.5053,66,77,78,8.05,NaN,1611953314
576,kihei,20.7850,-156.4656,75,64,40,19.57,US,1611953314


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [195]:
gmaps.configure(api_key=g_key)

In [196]:
locations = cities_df[['Lat','Lng']]
locations

,Lat,Lng
0,-33.6971,-53.4616
1,-46.1927,168.8643
2,-37.6167,-73.6500
3,50.8456,142.6550
4,-4.3246,15.3215
...,...,...
573,-38.6533,178.0042
574,53.3644,38.0694
575,-22.9575,14.5053
576,20.7850,-156.4656


In [197]:
humidity_cities = cities_df['Humidity']
humidity_cities
# cities_df['Humidity'].max()

0      68
1      80
2      94
3      91
4      69
       ..
573    71
574    97
575    77
576    64
577    83
Name: Humidity, Length: 578, dtype: int64

In [198]:
# Plot Heatmap
fig = gmaps.figure(center=(0,0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_cities, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5, opacity=0.6)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [199]:
cities_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp        int64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int32
dtype: object

In [216]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don’t contain all three conditions. You want to be sure the weather is ideal.

weather_fit_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]

weather_fit_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,saint-pierre,-21.3393,55.4781,78,65,0,4.61,RE,1611953194
43,filingue,14.3521,3.3168,75,18,0,9.55,NE,1611953198
99,saint-philippe,-21.3585,55.7679,78,65,0,4.61,RE,1611953211
217,gandorhun,7.5000,-11.8333,77,84,0,4.97,SL,1611953237
271,prado,-17.3411,-39.2208,78,77,0,8.68,BR,1611953249
345,kibala,9.1104,18.3463,77,33,0,3.11,TD,1611953265
390,atar,20.5169,-13.0499,71,21,0,9.26,MR,1611953275
420,emerald,-23.5333,148.1667,71,83,0,4.61,AU,1611953282
427,upington,-28.4478,21.2561,75,33,0,5.75,ZA,1611953284


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [217]:
hotel_df = weather_fit_df
weather_fit_df
weather_fit_df["Hotel Name"]=""
weather_fit_df

<ipython-input-217-5b416690cb9e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_fit_df["Hotel Name"]=""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,saint-pierre,-21.3393,55.4781,78,65,0,4.61,RE,1611953194,
43,filingue,14.3521,3.3168,75,18,0,9.55,NE,1611953198,
99,saint-philippe,-21.3585,55.7679,78,65,0,4.61,RE,1611953211,
217,gandorhun,7.5000,-11.8333,77,84,0,4.97,SL,1611953237,
271,prado,-17.3411,-39.2208,78,77,0,8.68,BR,1611953249,
345,kibala,9.1104,18.3463,77,33,0,3.11,TD,1611953265,
390,atar,20.5169,-13.0499,71,21,0,9.26,MR,1611953275,
420,emerald,-23.5333,148.1667,71,83,0,4.61,AU,1611953282,
427,upington,-28.4478,21.2561,75,33,0,5.75,ZA,1611953284,


In [218]:
from pprint import pprint

params={
    "radius":5000,
    "types":'lodging',
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in weather_fit_df.iterrows():
    
    params["location"] = f"{row['Lat']},{row['Lng']}"
    
    response = requests.get(base_url, params=params)
    json_response = response.json()
    
    try:
        weather_fit_df.loc[index, "Hotel Name"] = json_response["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")
        pass
    
weather_fit_df

C:\Users\shahp\anaconda31\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,saint-pierre,-21.3393,55.4781,78,65,0,4.61,RE,1611953194,Lindsey Hôtel
43,filingue,14.3521,3.3168,75,18,0,9.55,NE,1611953198,
99,saint-philippe,-21.3585,55.7679,78,65,0,4.61,RE,1611953211,"Chambres d'hôte ""La Trinité"""
217,gandorhun,7.5000,-11.8333,77,84,0,4.97,SL,1611953237,
271,prado,-17.3411,-39.2208,78,77,0,8.68,BR,1611953249,INN CORNER RIVER - PRADO - BAHIA - HOUSING AND...
345,kibala,9.1104,18.3463,77,33,0,3.11,TD,1611953265,
390,atar,20.5169,-13.0499,71,21,0,9.26,MR,1611953275,Oumou Elghoura
420,emerald,-23.5333,148.1667,71,83,0,4.61,AU,1611953282,The Irish Village
427,upington,-28.4478,21.2561,75,33,0,5.75,ZA,1611953284,River Place Manor


In [221]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [222]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))